In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

# Configurable settings
def load_datasets(sample_info_path, methylation_path):
    try:
        sample_info = pd.read_csv(sample_info_path)
        meth = pd.read_csv(methylation_path, index_col=0)
        return sample_info, meth
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None

# Load data
sample_info_path = input("Enter path for sample info CSV file: ")
methylation_path = input("Enter path for methylation data CSV file: ")
sample_info, meth = load_datasets(sample_info_path, methylation_path)

# Merge data
merged_data = sample_info.merge(meth, left_on='V1', right_index=True)
merged_data = merged_data.rename(columns={"V1": "SampleID"})
print("Merged data preview:", merged_data.head())

# Define configurable columns
covariates1 = ['age', 'bmi', 'sex', 'Life_PTS_severity', 'traumanum']  # customize as needed
covariates2 = ['age', 'bmi', 'sex', 'traumanum']
cpg_prefix = 'cg'  # Adjust to match CpG column naming in your dataset
cpg_columns = [col for col in merged_data.columns if col.startswith(cpg_prefix)]

# Define target columns
y_cvd = merged_data['CVD']
y_ptsdlife = merged_data['Life_PTS_severity']

# Normalize covariates
scaler = StandardScaler()
X_fil = merged_data[covariates1 + cpg_columns].copy()
X_fil[covariates1] = scaler.fit_transform(X_fil[covariates1])

Y_fil = merged_data[covariates2 + cpg_columns].copy()
Y_fil[covariates2] = scaler.fit_transform(Y_fil[covariates2])

# Analysis functions
def get_p_values_logit(X, y):
    X = sm.add_constant(X)  # Add constant for intercept
    model = sm.Logit(y, X).fit(disp=0)
    return model.pvalues, model.params, model.summary()

def get_p_values_ols(X, y):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit(disp=0)
    return model.pvalues, model.params, model.summary()

# Initialize result dictionaries
results = {'p_values_cvd': {}, 'p_values_ptsdlife': {}, 'coefficients_cvd': {}, 'coefficients_ptsdlife': {}}

# Process each CpG site
for cpg in cpg_columns:
    X_cvd_cpg = X_fil[covariates1].copy()
    X_cvd_cpg[cpg] = X_fil[cpg]
    
    if X_cvd_cpg[cpg].nunique() > 1:
        p_values, coefficients, _ = get_p_values_logit(X_cvd_cpg, y_cvd)
        results['p_values_cvd'][cpg] = p_values[cpg]
        results['coefficients_cvd'][cpg] = coefficients[cpg]

    X_ptsdlife_cpg = Y_fil[covariates2].copy()
    X_ptsdlife_cpg[cpg] = Y_fil[cpg]
    
    if X_ptsdlife_cpg[cpg].nunique() > 1:
        p_values, coefficients, _ = get_p_values_ols(X_ptsdlife_cpg, y_ptsdlife)
        results['p_values_ptsdlife'][cpg] = p_values[cpg]
        results['coefficients_ptsdlife'][cpg] = coefficients[cpg]

# Compile results and display significant CpG sites
results_combined = pd.DataFrame(results)
p_value_threshold = 0.05
significant_cpgs_cvd = [cpg for cpg, p_value in results['p_values_cvd'].items() if p_value < p_value_threshold]
significant_cpgs_ptsdlife = [cpg for cpg, p_value in results['p_values_ptsdlife'].items() if p_value < p_value_threshold]

